In [1]:
import sys
sys.path.append('../')

In [2]:
from scripts.utils import data_path, set_mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import glob
from tqdm import tqdm
import datetime


%matplotlib inline

set_mpl()

matplotlib settings set
matplotlib settings set


In [143]:
locations = pd.read_csv(data_path + 'Trentino_orig/line.csv')
#assign types [int, str, int]
locations.SQUAREID = locations.SQUAREID.astype(int)
locations.NR_UBICAZIONI = locations.NR_UBICAZIONI.astype(int)
locations.LINESET = locations.LINESET.astype(str)
locations = locations.set_index('SQUAREID')

energy_1 = pd.read_csv(data_path + 'Trentino_orig/SET-nov-2013.csv', names=['line', 'date', 'value'], parse_dates=['date'])
energy_2 = pd.read_csv(data_path + 'Trentino_orig/SET-dec-2013.csv', names=['line', 'date', 'value'], parse_dates=['date'])

energy = pd.concat([energy_1, energy_2])

energy = energy.pivot(index='date', columns='line', values='value')
energy = energy.resample('1H').mean()
energy.index = pd.to_datetime(energy.index)


locations.to_csv(data_path + 'Trentino_drive/line_location.csv')
energy.to_csv(data_path + 'Trentino_drive/line_energy.csv')

In [146]:
energy.index

DatetimeIndex(['2013-11-01 00:00:00', '2013-11-01 01:00:00',
               '2013-11-01 02:00:00', '2013-11-01 03:00:00',
               '2013-11-01 04:00:00', '2013-11-01 05:00:00',
               '2013-11-01 06:00:00', '2013-11-01 07:00:00',
               '2013-11-01 08:00:00', '2013-11-01 09:00:00',
               ...
               '2013-12-31 14:00:00', '2013-12-31 15:00:00',
               '2013-12-31 16:00:00', '2013-12-31 17:00:00',
               '2013-12-31 18:00:00', '2013-12-31 19:00:00',
               '2013-12-31 20:00:00', '2013-12-31 21:00:00',
               '2013-12-31 22:00:00', '2013-12-31 23:00:00'],
              dtype='datetime64[ns]', name='date', length=1464, freq='H')

In [118]:
#from plot.py by paper authors
parse = lambda x: datetime.datetime.fromtimestamp(float(x)/1000)

filelist =  glob.glob(data_path + 'Trentino_orig/telecom/*.txt')

aux_data = []

for file in tqdm(filelist[:]):
    aux = pd.read_csv(file, sep='\t', encoding="utf-8-sig", names=['CellID', 'datetime', 'countrycode', 'smsin', 
    'smsout', 'callin', 'callout', 'internet'], parse_dates=['datetime'], date_parser=parse)
    aux = aux.query('CellID in @locations.index & countrycode==39')
    aux.set_index('datetime', inplace=True)
    aux.drop('countrycode', axis=1, inplace=True)
    aux_data.append(aux)

aux_data = pd.concat(aux_data)
aux_data.sort_index(inplace=True)

aux_data

100%|██████████| 62/62 [12:28<00:00, 12.07s/it]


,CellID,smsin,smsout,callin,callout,internet
datetime,,,,,,
2014-01-01 00:00:00,10000,8.600870,13.861839,1.721833,3.424444,25.972920
2014-01-01 00:10:00,10000,8.227030,10.316980,2.125995,2.816777,22.230596
2014-01-01 00:20:00,10000,8.381257,5.646549,0.711157,1.277453,21.875093
2014-01-01 00:30:00,10000,7.217623,6.477903,0.678512,0.791203,24.124814
2014-01-01 00:40:00,10000,5.902213,3.109369,0.285036,0.292545,26.145020
...,...,...,...,...,...,...
2013-12-14 23:10:00,9999,0.289531,0.794115,0.156907,0.124489,16.734552
2013-12-14 23:20:00,9999,0.512851,0.806388,0.033362,NaN,17.700052
2013-12-14 23:30:00,9999,0.277126,0.690069,0.123545,0.158211,16.624514


In [147]:
aux_data.index

DatetimeIndex(['2013-11-01 00:00:00', '2013-11-01 00:00:00',
               '2013-11-01 00:00:00', '2013-11-01 00:00:00',
               '2013-11-01 00:00:00', '2013-11-01 00:00:00',
               '2013-11-01 00:00:00', '2013-11-01 00:00:00',
               '2013-11-01 00:00:00', '2013-11-01 00:00:00',
               ...
               '2014-01-01 23:50:00', '2014-01-01 23:50:00',
               '2014-01-01 23:50:00', '2014-01-01 23:50:00',
               '2014-01-01 23:50:00', '2014-01-01 23:50:00',
               '2014-01-01 23:50:00', '2014-01-01 23:50:00',
               '2014-01-01 23:50:00', '2014-01-01 23:50:00'],
              dtype='datetime64[ns]', name='datetime', length=17650780, freq=None)

In [121]:
aux_data.to_pickle(data_path + 'Trentino_drive/telecom.pkl_gz', compression='gzip')